In [1]:
from __future__ import print_function, division
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.tools as tls
print(__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)
from Genon import *

1.12.12


## The twelve examplar genes
Use Phenogenon to predict HPO and MOI for the twelve examplar genes.  
For details, please check our [preprint](https://www.biorxiv.org/content/early/2018/07/11/367292)

In [2]:
G = Genon()
GR = GenonResult()
G.init_genes()
for g in G.genes:
    G.genes[g].hpos = None
    G.genes[g].mode = None
predict = G.analyse(GR)
print(predict)

IMPG2 - Predicted mode is r
      - Number of patients for mode d: 57
      - Number of patients for mode r: 4
      - HPOs are predicted...
	HP:0000572
		HGF: 2.50223176222
		cadd_15_ratio: 0.999999999998
		MOI_score: 0.842383201981
	HP:0000512
		HGF: 1.82209993799
		cadd_15_ratio: 1.0
		MOI_score: 0.69002281569
	HP:0000556
		HGF: 1.50771136301
		cadd_15_ratio: 1.0
		MOI_score: -0.0769379507714
USH2A - Predicted mode is r
      - Number of patients for mode d: 350
      - Number of patients for mode r: 259
      - HPOs are predicted...
	HP:0000505
		HGF: 26.1990025573
		cadd_15_ratio: 0.505394793805
		MOI_score: 9.81779557857
	HP:0000510
		HGF: 20.8795225071
		cadd_15_ratio: 0.71618240851
		MOI_score: 3.77228278284
BBS1 - Predicted mode is r
     - Number of patients for mode d: 44
     - Number of patients for mode r: 10
     - HPOs are predicted...
	HP:0001133
		HGF: 5.61217434972
		cadd_15_ratio: 0.822019072852
		MOI_score: 5.01518497464
	HP:0000518
		HGF: 5.4760468501
		cadd_15_ra

## Phenogenon heatmaps
Phenogenon heatmaps for **_ABCA4_ - Macular dystrophy** (HP:0007754), recessive MOI and **_SCN1A_ - Seizures** (HP:0001250), dominant MOI  
The heat score (of a block) reflects the level of enrichment of patients with the phenotype among those who carry variants that can be mapped (to the block)

**Note** that the heat scales are different between the two maps.

In [3]:
G = Genon()
#G.steps = (5, 0.0001)
G.init_genes()
pm = G.genes['ABCA4'].patient_maps['r']
abca4 = G.phenogenon(G.genes['ABCA4'],pm,'HP:0007754')
pm = G.genes['SCN1A'].patient_maps['d']
scn1a = G.phenogenon(G.genes['SCN1A'],pm,'HP:0001250')
zmax = max(list(itertools.chain.from_iterable([i for i in scn1a])))
titles = ('<i>ABCA4</i>-Macular dystrophy','<i>SCN1A</i>-Seizures')
fig = tls.make_subplots(rows=1, cols=2, subplot_titles=titles, horizontal_spacing = 0.12)
log_transform = lambda x:x if np.isnan(x) else -math.log(x)
log_transform = np.vectorize(log_transform)
trace1 = {
    'z':log_transform(abca4),
    'x':np.arange(0,0.01,0.00025),
    'y':np.arange(0,60,5),
    'connectgaps': False,
    'type': 'heatmap',
    #'zsmooth': 'best',
    'showscale': True,
    'colorbar':{'x':0.44},
    #'colorscale':'Greys',
    #'zmax':8,
    'zmin':0,
    'zauto':False
}
trace2 = {
    'z':log_transform(scn1a),
    'x':np.arange(0,0.01,0.00025),
    'y':np.arange(0,60,5),
    'connectgaps': False,
    'type': 'heatmap',
    #'zsmooth': 'best',
    'showscale': True,
    #'colorscale':'Greys',
    'zmax':13,
    'zmin':0,
    'zauto':False
}
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig['layout']['yaxis1'].update(title={'text':'CADD_phred'})
fig['layout']['xaxis1'].update(title={'text':'GF'})
fig['layout']['xaxis2'].update(title={'text':'GF'})
fig['layout'].update(
    title={'text':'Phenogenon heatmap'},
    legend=dict(orientation="h"),
    height=300,
)
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



## Benchmark MOI prediction

In [4]:
def benchmark_moi(KEY,data):
    error = []
    G = Genon()
    G.init_genes()
    result = {i:{'error':0,'total':0} for i in np.arange(0,15.5,0.5)}
    for g in G.genes:
        # because GUCY2D can be either recessive or dominant MOI, exclude it from MOI benchmarking
        if g == 'GUCY2D': continue
        for hpo in G.genes[g].hpos:
            mode = None
            R = getattr(data,KEY)[g]['r'].get(hpo,0)
            D = getattr(data,KEY)[g]['d'].get(hpo,0)
            gs_R = data.HGF[g]['r'].get(hpo,0)
            gs_D = data.HGF[g]['d'].get(hpo,0)
            if R > D:
                mode = 'r'
            elif R < D:
                mode = 'd'
            if mode is None: continue
            for i in np.arange(0,min(math.floor(max(gs_R,gs_D)*2)/2,15.5),0.5):
                result[i]['total'] += 1
                if mode != G.genes[g].mode:
                    result[i]['error'] += 1
            if mode != G.genes[g].mode:
                for m in ('d','r'):
                    error.append((g,hpo,m,data.HGF[g][m].get(hpo,None),getattr(data,KEY)[g][m].get(hpo,None),getattr(data,KEY)[g]['r'].get(hpo,None)-getattr(data,KEY)[g]['d'].get(hpo,None)))
    return result, error
#good, total, good / total, error

In [7]:
result = defaultdict(dict)
result['Use hgf for MOI prediction'],error = benchmark_moi('HGF',predict)
result['Use MOI score for MOI prediction'],error = benchmark_moi('genon_hs',predict)

G = Genon()
G.use_af_for_recessive = True
G.steps = (5, 0.0001)
G.init_genes()
for g in G.genes:
    G.genes[g].hpos = None
    G.genes[g].mode = None
GR = GenonResult()
af4rec = G.analyse(GR)
G = Genon()
G.steps = (5, 0.0001)
G.combine_pvalues_method = 'fisher'
G.init_genes()
for g in G.genes:
    G.genes[g].hpos = None
    G.genes[g].mode = None
GR = GenonResult()
fish = G.analyse(GR)

result['Use AF for recessive'],error = benchmark_moi('genon_hs',af4rec)
result['Fisher'],error = benchmark_moi('genon_hs',fish)

In [8]:
# draw plot
import colorlover as cl
import copy
colors = {
    'Fisher': 'steelblue',
    'Use hgf for MOI prediction': 'red',
    'Use MOI score for MOI prediction': 'green',
    'Use AF for recessive': 'orange',
}
x = np.arange(0,15.5,0.5)
data = []
for k in ('Use AF for recessive','Use MOI score for MOI prediction','Use hgf for MOI prediction','Fisher'):
    trace = go.Scatter(
        x = x,
        y = [result[k][i]['error']/result[k][i]['total'] for i in x],
        name = k,
        mode = 'lines',
        line = dict(
            color = colors[k]
        )
    )
    data.append(trace)

layout = dict(
    title = {'text':'MOI prediction for each gene-HPO'},
    yaxis = dict(
        showgrid=False,
        title={'text':'Error rate'},
        zeroline=False,
    ),
    xaxis = dict(
        showgrid=False,
        title={'text':'hgf cutoff'},
        zeroline=False,
    )
)
iplot(dict(data=data,layout=layout))

In [9]:
GG = Genon()
GG.init_genes()
truth = GG.genes

In [10]:
# one can cache patient maps so that they don't get calculated every time
# Note that patient map for a gene is independent of N
patient_maps = dict()
predict = defaultdict(dict)
GR = GenonResult()
for n in range(100,5,-5):
    G = Genon()
    G.init_genes()
    G.N = n
    print(n)
    for g in G.genes:
        G.genes[g].hpos = None
    predict['optimal'][n] = G.analyse(GR,patient_maps=patient_maps)

100
95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10


In [12]:
patient_maps = dict()
GR = GenonResult()
for n in range(100,5,-5):
    G = Genon()
    G.use_af_for_recessive = True
    G.init_genes()
    G.N = n
    print(n)
    for g in G.genes:
        G.genes[g].hpos = None
    predict['use AF for recessive'][n] = G.analyse(GR,patient_maps=patient_maps)

100
95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10


In [13]:
patient_maps = dict()
GR = GenonResult()
for n in range(100,5,-5):
    G = Genon()
    G.combine_pvalues_method = 'fisher'
    G.init_genes()
    G.N = n
    print(n)
    for g in G.genes:
        G.genes[g].hpos = None
    predict['fisher'][n] = G.analyse(GR,patient_maps=patient_maps)

100
95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10


In [14]:
def predict_mode(gs,hr,vr,sr,gc,ph):
    '''
    predict inheritance mode
    return a number.
    Negative means dominant
    Positive means recessive
    '''
    vals = {'r':0,'d':0}
    for mode in ('r','d'):
        vals[mode] = max(gs[mode].values() or [0])
    return vals['r'] - vals['d']
    
patient_maps = dict()
GR = GenonResult()
for n in range(100,5,-5):
    G = Genon()
    G.predict_mode = predict_mode
    G.init_genes()
    G.N = n
    print(n)
    for g in G.genes:
        G.genes[g].hpos = None
    predict['hgf for MOI'][n] = G.analyse(GR,patient_maps=patient_maps)

100
95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10


In [15]:
# process truth first
from utils import *
class debugger:
    n = 0
    @classmethod
    def get_hpo_ancestors(cls,hpo_db,hpo):
        cls.n += 1
        return get_hpo_ancestors(hpo_db,hpo)
    
G = Genon()
processed_truth = {}
ancestors_cache = {}
hpos_to_mask = set(['HP:0000518'])#HP:0000518 is simply co-reported with most eye problems
# add some obvious hpos to genes
truth['ABCA4'].hpos += ['HP:0001098','HP:0000479','HP:0001133','HP:0001123','HP:0000505','HP:0012374','HP:0001103','HP:0004329','HP:0030453','HP:0030636','HP:0012373','HP:0012372','HP:0030466','HP:0000478']
truth['SCN1A'].hpos = ['HP:0001250','HP:0012638','HP:0012759','HP:0000707']
truth['USH2A'].hpos += ['HP:0000478','HP:0000580','HP:0012374','HP:0000598','HP:0012373','HP:0000365','HP:0000572','HP:0001123','HP:0030453','HP:0000580','HP:0030466','HP:0000505','HP:0001098','HP:0004328','HP:0012372','HP:0004329','HP:0007703','HP:0001133','HP:0000479'] 
truth['GUCY2D'].hpos += ['HP:0007754','HP:0012374','HP:0012373','HP:0007703','HP:0001133','HP:0000572','HP:0000479','HP:0004329','HP:0000496','HP:0000478','HP:0030466','HP:0012372','HP:0001098']
truth['PROM1'].hpos += ['HP:0012373','HP:0000478','HP:0012372','HP:0030453','HP:0012374','HP:0007703','HP:0004329','HP:0000505','HP:0001103','HP:0001098','HP:0012373','HP:0012373','HP:0000580','HP:0000479','HP:0000580','HP:0000572','HP:0000510']
truth['TERT'].hpos += ['HP:0011138','HP:0000271','HP:0000152','HP:0000153','HP:0000163','HP:0001597','HP:0001574','HP:0012647',"HP:0011024", "HP:0002012", "HP:0001438", "HP:0010978", "HP:0002715",'HP:0012649','HP:0012718','HP:0002242','HP:0004386','HP:0012719','HP:0002037','HP:0002250','HP:0001871','HP:0005561','HP:0001876','HP:0001877','HP:0012145']
truth['IMPG2'].hpos += ['HP:0000639','HP:0000572','HP:0030453','HP:0030453','HP:0000496','HP:0001098','HP:0004328','HP:0012373','HP:0007754']
truth['BBS1'].hpos += ['HP:0001133','HP:0001098','HP:0000478','HP:0012374','HP:0000479','HP:0012372','HP:0004329','HP:0030466','HP:0001123','HP:0007703','HP:0030453','HP:0000662','HP:0012373','HP:0000505','HP:0000580']
truth['CNGB1'].hpos += ['HP:0004329','HP:0012373','HP:0000572','HP:0000580','HP:0012374','HP:0030453','HP:0000478','HP:0012372','HP:0001133','HP:0000580','HP:0001098','HP:0000505','HP:0001123','HP:0007703','HP:0000479','HP:0004328','HP:0007754','HP:0001103']
truth['CRB1'].hpos += ['HP:0004329','HP:0012373','HP:0012372','HP:0000505','HP:0000572','HP:0001123','HP:0007703','HP:0001133','HP:0001103','HP:0000580','HP:0012374','HP:0000479','HP:0000580','HP:0000478','HP:0001098','HP:0030453','HP:0000478']
truth['RPGR'].hpos += ['HP:0000505','HP:0007703','HP:0001098','HP:0000510','HP:0000580','HP:0000479','HP:0012373','HP:0030466','HP:0001133','HP:0012374','HP:0012372','HP:0000478','HP:0001123','HP:0004329','HP:0030453']
truth['CERKL'].hpos += ['HP:0001123','HP:0012373','HP:0012374','HP:0000505','HP:0012372','HP:0000478','HP:0001133','HP:0007703','HP:0001098','HP:0000479','HP:0000580','HP:0004329',]

N = 0
for gene, v in truth.items():
    processed_truth[gene] = {
        'mode':v.mode,
        'hpos':set(v.hpos)
    }
    for hpo in G.phs:
        if hpo in processed_truth[gene]['hpos']: continue
        hpos_to_check = [h for h in v.hpos if G.phs[h] >= G.phs[hpo]]
        for h in hpos_to_check:
            N += 1
            # setdefault is not lazy evaluating?? Lame!
            ancestors = ancestors_cache.get(hpo) or ancestors_cache.setdefault(hpo, set([i['id'][0] for i in debugger.get_hpo_ancestors(G.hpo_db, hpo)]))
            if h in ancestors:
                processed_truth[gene]['hpos'].update([hpo])
                break

In [42]:
def benchmark(truth,bundle,Range, Max=False):
    '''
    Given truth, benchmark results in bundle
    returns
    overall mistakes, tests number for both modes and HPOs
    and detailed down for each gene for HPOs, and each mode for HPOs
    results:
        method:
            gene:
                test:
                    [{
                    N:100
                    mistakes:
                    number_of_tests:
                    }]
        ...
                
        
    '''
    result = defaultdict(lambda:defaultdict(lambda:defaultdict(list)))
    for r in Range:
        for method, predict in bundle.items():
            # mode
            for gene in predict[r].genon_hs:
                mode = 'u'
                if method == 'hgf for MOI':
                    m = max(predict[r].HGF[gene]['r'].values() or [0]) - max(predict[r].HGF[gene]['d'].values() or [0])
                else:
                    m = predict[r].predicted_moi[gene]
                if m > 0:
                    mode = 'r'
                elif m < 0:
                    mode = 'd'
                if mode != truth[gene]['mode']:
                    mistakes = 1
                else:
                    mistakes = 0
                result[method][gene]['moi'].append({
                    'N':r,
                    'mistakes':mistakes,
                    'number_of_tests':1,
                })
                
                # using predicted mode for hpo test
                value = predict[r].HGF[gene]
                if mode in value:
                    the_h = None
                    if Max:
                        mx = max(value[mode].values())
                        for h,v in value[mode].items():
                            if v == mx:
                                the_h = h
                                break
                        mistakes,NOT = set(),1
                        if the_h not in truth[gene]['hpos'] and the_h not in hpos_to_mask:
                            mistakes = set([the_h])
                    else:
                        hs = set(predict[r].ph[gene][mode]) - hpos_to_mask
                        mistakes = hs - truth[gene]['hpos']
                        NOT = len(hs)
                else:
                    mistakes, NOT = set(),0
                result[method][gene]['hpo'].append({
                    'N':r,
                    'mistakes':len(mistakes),
                    'number_of_tests':NOT,
                    'wrong_hpos':mistakes,
                })
                
                # using true mode for hpo test
                t_mode = truth[gene]['mode']
                if Max:
                        mx = max(value[t_mode].values())
                        for h,v in value[t_mode].items():
                            if v == mx:
                                the_h = h
                                break
                        mistakes,NOT = set(),1
                        if the_h not in truth[gene]['hpos'] and the_h not in hpos_to_mask:
                            mistakes = set([the_h])
                else:
                    hs = set(predict[r].ph[gene][t_mode]) - hpos_to_mask
                    mistakes = hs - truth[gene]['hpos']
                    NOT = len(hs)
                result[method][gene]['hpo_using_true_mode'].append({
                    'N':r,
                    'mistakes':len(mistakes),
                    'number_of_tests':NOT,
                    'wrong_hpos':mistakes,
                })
    return result

In [17]:
import colorlover as cl
import copy
colorScales = copy.copy(cl.scales['12']['qual']['Paired'])
colorScales[2:6] = colorScales[4:6] + colorScales[2:4]
# swap second and third pair to make graph clearer
def plot_benchmark(bm,keys,Range):
    x = list(Range)
    # draw moi, hpo, hpo_using_true_mode separately
    tp_dict = dict(
        hpo = 'Predict HPO',
        hpo_using_true_mode = 'Predict HPO using true MOI',
        moi = 'Predict MOI',
    )
    for tp in ('hpo','hpo_using_true_mode','moi'):
        ind = 0
        traces = []
        offset = 0
        unit,width = 0.8,0.8
        for method in keys:
            data = bm[method]
            y,y_error,y_correct = [None] * len(x),[None] * len(x),[None] * len(x)
            for i in range(len(x)):
                error,total = 0,0
                for gene in data:
                    error += data[gene][tp][i]['mistakes']
                    total += data[gene][tp][i]['number_of_tests']
                y[i] = error / total if total else 0
                y_error[i] = error
                y_correct[i] = total - error
            trace_line = go.Scatter(
                x = x,
                y = y,
                mode = 'lines',
                name = '{}-error rate'.format(method),
                line = dict(
                    color = colorScales[ind+1],
                    #width = 4,
                #    dash = dash_dict[tp],
                )
            )
            trace_bar_error = go.Bar(
                x = x,
                y = y_error,
                name = '{}-error count'.format(method),
                yaxis='y2',
                base = 0,
                width = width,
                offset = offset,
                marker = dict(color = colorScales[ind+1]),
            )
            trace_bar_correct = go.Bar(
                x = x,
                y = y_correct,
                name = '{}-correct count'.format(method),
                yaxis='y2',
                base = y_error,
                width = width,
                offset = offset,
                marker = dict(color = colorScales[ind]),
            )
            traces.extend([trace_bar_error,trace_bar_correct,trace_line,])
            offset += unit
            ind += 2
        layout = dict(
                title = {'text':tp_dict[tp]},
                xaxis = dict(
                    title = {'text':'HPO NP cutoff'},
                ),
                yaxis2=dict(
                        title={'text':'count'},
                        overlaying='y',
                        showgrid=False,
                ),    
                yaxis = dict(
                    title = {'text':'Error rate'},
                    showgrid=False,
                    side='right',
                    zeroline = False,
                ),
                
                barmode='stack',
                legend=dict(
                    x = 1.15,
                ),
            )
        fig = dict(data=traces, layout=layout)
        iplot(fig)

In [34]:
print(predict['optimal'][100].predicted_moi)

{'IMPG2': -0.17473175611160574, 'USH2A': 6.0651918497582011, 'BBS1': 4.0195625923311376, 'RPGR': 0.38417050923670537, 'CRB1': 4.8597643554273988, 'ABCA4': 1.3553742742017381, 'CNGB1': 3.0088599391141719, 'CERKL': 3.5245270379119304, 'PROM1': -0.94127105846902415, 'SCN1A': -58.395518026776692, 'GUCY2D': -0.27321489227348938, 'TERT': -3.5900727225362985}


In [39]:
for method in bm:
    for gene in bm[method]:
        for test,data in bm[method][gene].items():
            for d in data:
                if d['mistakes']:
                    print(method,gene,test)
                    print(d)

optimal IMPG2 moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 100}
optimal IMPG2 moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 15}
optimal IMPG2 moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 10}
optimal IMPG2 hpo
{'wrong_hpos': set(['HP:0004430', 'HP:0000951']), 'mistakes': 2, 'number_of_tests': 4, 'N': 100}
optimal IMPG2 hpo
{'wrong_hpos': set(['HP:0004322', 'HP:0004430', 'HP:0001596', 'HP:0000951', 'HP:0008404']), 'mistakes': 5, 'number_of_tests': 7, 'N': 15}
optimal IMPG2 hpo
{'wrong_hpos': set(['HP:0004322', 'HP:0001010', 'HP:0008404', 'HP:0004430', 'HP:0001511', 'HP:0001596']), 'mistakes': 6, 'number_of_tests': 10, 'N': 10}
optimal CRB1 hpo
{'wrong_hpos': set(['HP:0001511']), 'mistakes': 1, 'number_of_tests': 6, 'N': 15}
optimal CRB1 hpo
{'wrong_hpos': set(['HP:0001511']), 'mistakes': 1, 'number_of_tests': 6, 'N': 10}
optimal CRB1 hpo_using_true_mode
{'wrong_hpos': set(['HP:0001511']), 'mistakes': 1, 'number_of_tests': 6, 'N': 15}
optimal CRB1 hpo_using_true_mode
{'wrong_hpos': s

hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 50}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 45}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 40}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 35}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 30}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 25}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 20}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 15}
hgf for MOI GUCY2D moi
{'mistakes': 1, 'number_of_tests': 1, 'N': 10}
fisher IMPG2 hpo
{'wrong_hpos': set(['HP:0004430']), 'mistakes': 1, 'number_of_tests': 4, 'N': 100}
fisher IMPG2 hpo
{'wrong_hpos': set(['HP:0004430']), 'mistakes': 1, 'number_of_tests': 4, 'N': 95}
fisher IMPG2 hpo
{'wrong_hpos': set(['HP:0004430']), 'mistakes': 1, 'number_of_tests': 4, 'N': 90}
fisher IMPG2 hpo
{'wrong_hpos': set(['HP:0004430']), 'mistakes': 1, 'num

In [61]:
bm['optimal']['GUCY2D']['hpo_using_true_mode']

[{'N': 100, 'mistakes': 0, 'number_of_tests': 2, 'wrong_hpos': set()},
 {'N': 95, 'mistakes': 0, 'number_of_tests': 2, 'wrong_hpos': set()},
 {'N': 90, 'mistakes': 0, 'number_of_tests': 2, 'wrong_hpos': set()},
 {'N': 85, 'mistakes': 0, 'number_of_tests': 3, 'wrong_hpos': set()},
 {'N': 80, 'mistakes': 0, 'number_of_tests': 3, 'wrong_hpos': set()},
 {'N': 75, 'mistakes': 0, 'number_of_tests': 3, 'wrong_hpos': set()},
 {'N': 70, 'mistakes': 0, 'number_of_tests': 2, 'wrong_hpos': set()},
 {'N': 65, 'mistakes': 0, 'number_of_tests': 2, 'wrong_hpos': set()},
 {'N': 60, 'mistakes': 0, 'number_of_tests': 2, 'wrong_hpos': set()},
 {'N': 55, 'mistakes': 0, 'number_of_tests': 3, 'wrong_hpos': set()},
 {'N': 50, 'mistakes': 0, 'number_of_tests': 3, 'wrong_hpos': set()},
 {'N': 45, 'mistakes': 0, 'number_of_tests': 3, 'wrong_hpos': set()},
 {'N': 40, 'mistakes': 0, 'number_of_tests': 4, 'wrong_hpos': set()},
 {'N': 35, 'mistakes': 0, 'number_of_tests': 4, 'wrong_hpos': set()},
 {'N': 30, 'mistake

In [43]:
Range = range(100,5,-5)
bm = benchmark(processed_truth,predict,Range,Max=False)

In [46]:
keys = ['fisher', 'hgf for MOI', 'optimal','use AF for recessive']
plot_benchmark(bm,keys,Range)